In [1]:
#!pip install selenium
#!pip install webdriver-manager
#!pip install tqdm

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import time
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
#service = Service(ChromeDriverManager().install())
#driver = webdriver.Chrome(service=service)

op = webdriver.ChromeOptions()
op.add_argument('headless')
driver = webdriver.Chrome(options=op)

url = "https://www.mercadolivre.com.br/ofertas?page="
produtos = []
preco = []
desconto = []

for i in tqdm_notebook(range(1, 291), desc='Loading'):
    driver.get(url + str(i))
    driver.implicitly_wait(1)
    
    tituloProduto = driver.find_elements(By.CLASS_NAME, "promotion-item__title")
    precoProduto = driver.find_elements(By.CLASS_NAME, "andes-money-amount__fraction")
    produtoDesconto = driver.find_elements(By.CLASS_NAME, "andes-money-amount__discount")
    
    for x in tituloProduto:
        produtos.append(x.text)

    for x in precoProduto:
        preco.append(x.text)
        
    for x in produtoDesconto:
        desconto.append(x.text)

df = pd.DataFrame()

df['produto'] = pd.Series(produtos)
df['preco'] = pd.Series(preco)
df['desconto'] = pd.Series(desconto)
df.dropna(how='any', inplace=True)

Loading:   0%|          | 0/290 [00:00<?, ?it/s]

In [2]:
#df.to_pickle('df.pkl')
df = pd.read_pickle('df.pkl')

In [3]:
for p in tqdm_notebook(df['preco'], desc='Processing'):
    df['preco'] = pd.Series(df['preco'], dtype="string").str.replace('.', '').astype(int)
    
for d in tqdm_notebook(df['desconto'], desc='Processing'):
    df['desconto'] = pd.Series(df['desconto'], dtype="string").str.replace('% OFF', '').astype(int)

Processing:   0%|          | 0/13640 [00:00<?, ?it/s]

Processing:   0%|          | 0/13640 [00:00<?, ?it/s]

In [5]:
price_mean = df.preco.mean()
disc_mean = df.desconto.mean()

print('O valor médio é de R$%.2f' % price_mean, 'e o desconto médio é de %.2f%%' % disc_mean)

O valor médio é de R$947.08 e o desconto médio é de 25.40%


In [7]:
df.sort_values(by='preco', ascending=False).head(10)

,produto,preco,desconto
2734,Robô Aspirador Wap Robot W90 3 Modos De Limpeza Fw009026,7549,12
1156,Braço Transversal Traseiro Direito Honda Civic 91 A 2001,7199,15
586,Pistola Pintura Pulverizadora Tinta Paint Com Bico De Metal,6799,26
2364,Samsung Galaxy A03 Core Dual SIM 32 GB black 2 GB RAM,6599,41
2733,"Smart TV Philips 6900 Series 43PFG6917/78 LED Full HD 43"" 110V/240V",6419,31
2189,Impressora Térmica Epson Tm-t20x Não Fiscal Usb - C31ch26031 Cor Preto Voltagem Bivolt,6137,30
2363,Vinho Casillero Del Diablo 2019 Adega Concha Y Toro 750 Ml,5599,10
585,"Rifle M4a1 + Pistola Airsoft Vigor V307 + Esferas 0,12g Loi",5499,42
2188,Máquina Solda Inversora Mig 150a 127v Sem Gás Tig Lift,5099,12
2278,Impressora a cor multifuncional HP Deskjet Ink Advantage 2774 com wifi preta 100V/240V,5057,25
